
# Cardiovascular Stent Simulation


This example problem shows how to simulate stent-artery interaction during and after stent
placement in an occluded artery.
The analysis exposes advanced modeling techniques using PyMAPDL such as:
* Contact
* Element birth and death
* Mixed u-P formulation
* Nonlinear stabilization

This example is inspired from the model and analysis defined in Chapter 25 of the Mechanical
APDL Technology Showcase Manual.

## Starting MAPDL as a service

In [1]:
# starting MAPDL as a service and importing an external model
from ansys.mapdl.core import launch_mapdl

# start MAPDL as a service
mapdl = launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise
MAPDL Version:       22.2
ansys.mapdl Version: 0.64.dev0



# Setting up the model

### Defining material properties<br>

In [2]:
# define 316L Stainless steel
mapdl.prep7()
mapdl.mptemp()
mapdl.mptemp(sloc="1", t1="0")
mapdl.mpdata(lab="EX", mat="1", c1="200e3")
mapdl.mpdata(lab="PRXY", mat="1", c1="0.3")
mapdl.mptemp()
mapdl.mptemp(sloc="1", t1="0")
mapdl.mpdata(lab="DENS", mat="1", c1="8000e-9")

PROPERTY TABLE DENS  MAT=      1  NUM. POINTS=  1
 SLOC=   1    0.8000000E-05

### Defining element types

In [3]:
# for straight line segments
mapdl.et(itype="1", ename="beam189")
mapdl.sectype(secid="1", type_="beam", subtype="csolid")
mapdl.secdata(val1=0.05)

# for arcs
mapdl.et(itype="2", ename="beam189")
mapdl.sectype(secid="2", type_="beam", subtype="csolid")
mapdl.secdata(val1=0.05)

SECTION ID NUMBER IS:            2
   BEAM SECTION TYPE IS:     Circular Solid  
   BEAM SECTION NAME IS:             
   COMPUTED BEAM SECTION DATA SUMMARY:
    Area                 = 0.78479E-02
    Iyy                  = 0.48986E-05
    Iyz                  =-0.66174E-22
    Izz                  = 0.48986E-05
    Warping Constant     = 0.46449E-40
    Torsion Constant     = 0.97972E-05
    Centroid Y           =-0.26983E-18
    Centroid Z           =-0.48569E-18
    Shear Center Y       = 0.36554E-20
    Shear Center Z       = 0.18437E-17
    Shear Correction-xy  = 0.85691    
    Shear Correction-yz  = 0.35124E-15
    Shear Correction-xz  = 0.85691    
                 
    Beam Section is offset to CENTROID of cross section

### Defining 5-parameter Mooney-Rivlin hyperelastic artery material model

In [4]:
c10 = 18.90e-3
c01 = 2.75e-3
c20 = 590.43e-3
c11 = 857.2e-3
nu1 = 0.49
dd = 2 * (1 - 2 * nu1) / (c10 + c01)

In [5]:
mapdl.tb(lab="hyper", mat="2", npts="5", tbopt="mooney")
mapdl.tbdata(stloc="1", c1="c10", c2="c01", c3="c20", c4="c11", c6="dd")

DATA FOR  HYPE  TABLE FOR MATERIAL   2 AT TEMPERATURE=  0.0000    
 LOC=  1 7.88861e-31     7.88861e-31     7.88861e-31     7.88861e-31     0.00000e+00     7.88861e-31

### Defining linear elastic material model for stiff calcified plaque

In [6]:
mapdl.mp(lab="EX", mat="3", c0=".00219e3")
mapdl.mp(lab="NUXY", mat="3", c0="0.49")

MATERIAL          3     NUXY =  0.4900000

### Define Solid185 element type to mesh both the artery and plaque

In [7]:
# for artery
mapdl.et(itype="9", ename="SOLID185")
mapdl.keyopt(
    itype="9", knum="6", value="1")  # Use mixed u-P formulation to avoid locking
mapdl.keyopt(itype="9", knum="2", value="3")  # Use Simplified Enhanced Strain method

# for plaque
mapdl.et(itype="16", ename="SOLID185")
mapdl.keyopt(itype="16", knum="2", value="0")  # Use B-bar

ELEMENT TYPE      16 IS SOLID185     3-D 8-NODE STRUCTURAL SOLID 
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 CURRENT NODAL DOF SET IS  UX    UY    UZ    ROTX  ROTY  ROTZ
  THREE-DIMENSIONAL MODEL

### Defining settings to model the stent, the artery and the plaque

Use force-distributed boundary constraints on 2 sides of artery wall to allow
for radial expansion of tissue without rigid body motion.

#### Settings for MPC Surface-based, force-distributed contact on proximal plane parallel to x-y plane

In [8]:
mapdl.mat("2")
mapdl.r(nset="3")
mapdl.real(nset="3")
mapdl.et(itype="3", ename="170")
mapdl.et(itype="4", ename="174")
mapdl.keyopt(itype="4", knum="12", value="5")
mapdl.keyopt(itype="4", knum="4", value="1")
mapdl.keyopt(itype="4", knum="2", value="2")
mapdl.keyopt(itype="3", knum="2", value="1")
mapdl.keyopt(itype="3", knum="4", value="111111")
mapdl.type(itype="3")

mapdl.mat("2")
mapdl.r(nset="4")
mapdl.real(nset="4")
mapdl.et(itype="5", ename="170")
mapdl.et(itype="6", ename="174")
mapdl.keyopt(itype="6", knum="12", value="5")
mapdl.keyopt(itype="6", knum="4", value="1")
mapdl.keyopt(itype="6", knum="2", value="2")
mapdl.keyopt(itype="5", knum="2", value="1")
mapdl.keyopt(itype="5", knum="4", value="111111")
mapdl.type(itype="5")

ELEMENT TYPE SET TO         5

#### Settings for standard contact between stent and inner plaque wall contact surface

In [9]:
mapdl.mp(lab="MU", mat="1", c0="0")
mapdl.mat("1")
mapdl.mp(lab="EMIS", mat="1", c0="7.88860905221e-31")
mapdl.r(nset="6")
mapdl.real(nset="6")
mapdl.et(itype="10", ename="170")
mapdl.et(itype="11", ename="177")
mapdl.r(nset="6", r3="1.0", r4="1.0", r5="0")
mapdl.rmore(r9="1.0E20", r10="0.0", r11="1.0")
mapdl.rmore(r7="0.0", r8="0", r9="1.0", r10="0.05", r11="1.0", r12="0.5")
mapdl.rmore(r7="0", r8="1.0", r9="1.0", r10="0.0")
mapdl.keyopt(itype="11", knum="5", value="0")
mapdl.keyopt(itype="11", knum="7", value="1")
mapdl.keyopt(itype="11", knum="8", value="0")
mapdl.keyopt(itype="11", knum="9", value="0")
mapdl.keyopt(itype="11", knum="10", value="2")
mapdl.keyopt(itype="11", knum="11", value="0")
mapdl.keyopt(itype="11", knum="12", value="0")
mapdl.keyopt(itype="11", knum="2", value="3")
mapdl.keyopt(itype="10", knum="5", value="0")

ELEMENT TYPE      10 IS TARGE170     3-D TARGET SEGMENT          
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 CURRENT NODAL DOF SET IS  UX    UY    UZ    ROTX  ROTY  ROTZ
  THREE-DIMENSIONAL MODEL

#### Settings for MPC based, force-distributed constraint on proximal stent nodes

In [10]:
mapdl.mat("1")
mapdl.r(nset="7")
mapdl.real(nset="7")
mapdl.et(itype="12", ename="170")
mapdl.et(itype="13", ename="175")
mapdl.keyopt(itype="13", knum="12", value="5")
mapdl.keyopt(itype="13", knum="4", value="1")
mapdl.keyopt(itype="13", knum="2", value="2")
mapdl.keyopt(itype="12", knum="2", value="1")
mapdl.keyopt(itype="12", knum="4", value="111111")
mapdl.type(itype="12")

ELEMENT TYPE SET TO        12

#### Settings for MPC based, force-distributed constraint on distal stent nodes

In [11]:
mapdl.mat("1")
mapdl.r(nset="8")
mapdl.real(nset="8")
mapdl.et(itype="14", ename="170")
mapdl.et(itype="15", ename="175")
mapdl.keyopt(itype="15", knum="12", value="5")
mapdl.keyopt(itype="15", knum="4", value="1")
mapdl.keyopt(itype="15", knum="2", value="2")
mapdl.keyopt(itype="14", knum="2", value="1")
mapdl.keyopt(itype="14", knum="4", value="111111")
mapdl.type(itype="14")

ELEMENT TYPE SET TO        14

### Reading the geometry file

In [12]:
mapdl.cdread(option="db", fname="stent", ext="cdb")
mapdl.allsel(labt="all")
mapdl.finish()

***** ROUTINE COMPLETED *****  CP =         0.844

## Static Analysis

Run static analysis

In [13]:
# enter solution processor and define analysis settings
mapdl.run("/solu")
mapdl.antype(antype="0")
mapdl.nlgeom(key="on")

LARGE DEFORMATION ANALYSIS

### Apply Load Step 1: Balloon angioplasty of the artery to expand it past the radius of the stent - IGNORE STENT

In [14]:
mapdl.nsubst(nsbstp="20", nsbmx="20")
mapdl.nropt(option1="full")
mapdl.cncheck(option="auto")
mapdl.esel(type_="s", item="type", vmin="11")
mapdl.cm(cname="contact2", entity="elem")
mapdl.ekill(elem="contact2")  # Kill contact elements in stent-plaque contact 
                              #pair so that the stent is ignored in the first loadstep
mapdl.nsel(type_="s", item="loc", comp="x", vmin="0", vmax="0.01e-3")
mapdl.nsel(type_="r", item="loc", comp="y", vmin="0", vmax="0.01e-3")
mapdl.d(node="all", lab="all")
mapdl.allsel()

mapdl.sf(nlist="load", lab="pres", value="10e-2")  # Apply 0.1 Pa/mm^2 pressure to inner plaque wall
mapdl.allsel()
mapdl.nldiag(label="cont", key="iter")
mapdl.solve()
mapdl.save()

ALL CURRENT MAPDL DATA WRITTEN TO FILE NAME= file.db
  FOR POSSIBLE RESUME FROM THIS POINT

### Apply Load Step 2: Reactivate contact between stent and plaque

In [15]:
mapdl.ealive(elem="contact2")
mapdl.allsel()

mapdl.nsubst(nsbstp="2", nsbmx="2")
mapdl.save()
mapdl.solve()

*****  MAPDL SOLVE    COMMAND  *****

 *** NOTE ***                            CP =      97.969   TIME= 10:09:39
 Present time 0 is less than or equal to the previous time.  Time will   
 default to 2.                                                           

 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2022 R2          22.2     ***
 Ansys Mechanical Enterprise                       
 00000000  VERSION=WINDOWS x64   10:09:39  JAN 09, 2023 CP=     98.312

                                                                               



                      L O A D   S T E P   O P T I O N S

   LOAD STEP NUMBER. . . . . . . . . . . . . . . .     2
   TIME AT END OF THE LOAD STEP. . . . . . . . . .  2.0000    
   AUTOMATIC TIME STEPPING . . . . . . . . . . . .    ON
      INITIAL NUMBER OF SUBSTEPS . . . . . . . . .     2
      MAXIMUM NUMBER OF SUBSTEPS . . . . . . . . .     2
      MINIMUM NUMBER OF SUBSTEPS . . . . . . . . .     1
   MAXIMUM NUMBER OF EQUILIBRIUM ITERATIONS. .

### Apply Load Step 3

In [16]:
mapdl.nsubst(nsbstp="1", nsbmx="1", nsbmn="1")
mapdl.solve()

*****  MAPDL SOLVE    COMMAND  *****

 *** NOTE ***                            CP =     120.359   TIME= 10:10:01
 Present time 0 is less than or equal to the previous time.  Time will   
 default to 3.                                                           

 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2022 R2          22.2     ***
 Ansys Mechanical Enterprise                       
 00000000  VERSION=WINDOWS x64   10:10:01  JAN 09, 2023 CP=    120.609

                                                                               



                      L O A D   S T E P   O P T I O N S

   LOAD STEP NUMBER. . . . . . . . . . . . . . . .     3
   TIME AT END OF THE LOAD STEP. . . . . . . . . .  3.0000    
   AUTOMATIC TIME STEPPING . . . . . . . . . . . .    ON
      INITIAL NUMBER OF SUBSTEPS . . . . . . . . .     1
      MAXIMUM NUMBER OF SUBSTEPS . . . . . . . . .     1
      MINIMUM NUMBER OF SUBSTEPS . . . . . . . . .     1
   MAXIMUM NUMBER OF EQUILIBRIUM ITERATIONS. .

### Apply Load Step 4: Apply blood pressure (13.3 kPa) load to inner wall of plaque and allow the stent to act as a scaffold

In [17]:
mapdl.nsubst(nsbstp="300", nsbmx="3000", nsbmn="30")
mapdl.sf(nlist="load", lab="pres", value="13.3e-3")
mapdl.allsel()

SELECT ALL ENTITIES OF TYPE= ALL  AND BELOW

### Apply stabilization with energy option

In [18]:
mapdl.stabilize(key="const", method="energy", value="0.1")

NONLINEAR STABILIZATION CONTROL:
 KEY=CONS   METHOD=ENER   VALUE= 0.10000       SUBSTPOPT=NO     FORCELIMIT= 0.20000

# Solve the model

In [19]:
mapdl.solve()
mapdl.save()
mapdl.finish()

FINISH SOLUTION PROCESSING


 ***** ROUTINE COMPLETED *****  CP =      1802.203

# Post-processing the results

This section illustrates the use of PyDPF-Core to post-process the results.

In [20]:
from ansys.dpf import core as dpf
from ansys.dpf.core import operators as ops

## Mesh of the model

In [21]:
model = dpf.Model(mapdl.result_file)
ds = dpf.DataSources(mapdl.result_file)

In [22]:
mesh = model.metadata.meshed_region
mesh.plot(vtk_export=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Computed displacements of the model

In [77]:
# Collecting the computed displacement
u = model.results.displacement(time_scoping=[4]).eval()

u[0].plot(deform_by = u[0])

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Von Mises stress

In [78]:
# Collecting the computed stress
s_op = model.results.stress(time_scoping=[3])
s_op.inputs.requested_location.connect(dpf.locations.nodal)
s = s_op.eval()

# Calculating Von Mises stress
s_VM = dpf.operators.invariant.von_mises_eqv_fc(fields_container=s)
s_VM_plot = s_VM.eval()

s_VM_plot[0].plot(deform_by = u[0])

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [79]:
s_VM_plot[0].name

'stress_3.s_eqv'

In [25]:
# vtk_export =model.operator("vtk_export")
# vtk_export.inputs.connect(r's_VM.vtk')
# vtk_export.inputs.connect(mesh)
# vtk_export.inputs.connect(s_VM.outputs)
# vtk_export.eval()

## Computed displacements of the stent

In [26]:
# Creating the mesh associated to the stent
esco = mesh.named_selection("STENT")
print(esco)

# Transposing elemental location to nodal one
op = dpf.operators.scoping.transpose()
op.inputs.mesh_scoping.connect(esco)
op.inputs.meshed_region.connect(mesh)
op.inputs.inclusive.connect(1)
nsco = op.eval()
print(nsco)

DPF  Scoping: 
  with Elemental location and 1760 entities

DPF  Scoping: 
  with Nodal location and 3520 entities



In [27]:
# Collecting the computed displacements of the stent
u_stent = model.results.displacement(mesh_scoping=nsco, time_scoping=[4])
u_stent = u_stent.outputs.fields_container()

# Linking the stent mesh to the global one
op = dpf.operators.mesh.from_scoping() # operator instantiation
op.inputs.scoping.connect(nsco)
op.inputs.inclusive.connect(1)
op.inputs.mesh.connect(mesh)
mesh_sco = op.eval()
u_stent[0].meshed_region = mesh_sco

u_stent[0].plot(deformed_by=u_stent[0])

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Exporting the plot to vtk files

In [28]:
grid_mesh = mesh.grid
grid_mesh.save('mesh.vtk')

In [29]:
grid_mesh_sco = mesh_sco.grid
grid_mesh_sco.save('mesh_sco.vtk')

In [30]:
vtk_export = model.operator("vtk_export")
vtk_export.inputs.connect(0)
vtk_export.inputs.connect(r'u.vtk')
vtk_export.inputs.connect(mesh)
vtk_export.inputs.connect(u)
vtk_export.eval()

In [31]:
vtk_export =model.operator("vtk_export")
vtk_export.inputs.connect(0)
vtk_export.inputs.connect(r's_VM.vtk')
vtk_export.inputs.connect(mesh)
vtk_export.inputs.connect(s_VM)
vtk_export.eval()

In [32]:
vtk_export = model.operator("vtk_export")
vtk_export.inputs.connect(r'u_stent.vtk')
vtk_export.inputs.connect(mesh_sco)
vtk_export.inputs.connect(u_stent)
vtk_export.eval()

In [88]:
# Saving the label name

f = open("data_label.dat", "w")
f.write(u[0].name + '\n')
f.write(s_VM_plot[0].name + '\n')
f.write(u_stent[0].name + '\n')

17

## Correcting the exporting files

In [109]:
f = open("u.vtk", "r")
l = len(f.readlines())
f1 = open("u.vtk", "r")
f2 = open("u_corr.vtk", "w")
a = f.readline() 
f2.write(a)
b = 'nan nan nan\n'
c = 'VECTORS'
for i in range(l): 
    a = f1.readline() 
    if a == b: print(2_stress_3.s_eqv_())
        a = '0 0 0\n'
    elif a[:len(c)] == c : # renamming the data
        a = 'VECTORS U double\n'
    f2.write(a)
f.close() 


In [114]:
f = open("s_VM.vtk", "r")
l = len(f.readlines())
f1 = open("s_VM.vtk", "r")
f2 = open("s_VM_corr.vtk", "w")
a = f.readline() 
f2.write(a)
b = 'nan nan nan\n'
c = 'SCALARS 2_'
for i in range(l): 
    a = f1.readline() 
    if a == b: # nan value not supported
        a = '0 0 0\n'
    elif a[:len(c)] == c : # renamming the data
        a = 'SCALARS S_VM double 1\n'
    f2.write(a)
f.close() 

In [108]:
f = open("u_stent.vtk", "r")
l = len(f.readlines())
f1 = open("u_stent.vtk", "r")
f2 = open("u_stent_corr.vtk", "w")
a = f.readline() 
f2.write(a)
b = 'nan nan nan\n'
c = 'VECTORS'
for i in range(l): 
    a = f1.readline() 
    if a == b: # nan value not supported
        a = '0 0 0\n'
    elif a[:len(c)] == c : # renamming the data
        a = 'VECTORS U_STENT double\n'
    f2.write(a)
f.close() 


In [33]:
#TO DO: 
# - automate the changes: nan -> 0

## Plot using the VTK files

In [34]:
import pyvista

In [35]:
file = "mesh.vtk"
mesh_file = pyvista.read(file)

file = "mesh_sco.vtk"
mesh_sco_file = pyvista.read(file)

pl = pyvista.Plotter(shape=(1, 2))
pl.subplot(0, 0)
pl.add_mesh(mesh_file, cmap="jet", show_scalar_bar=False, show_edges=True)
pl.add_text("Mesh", color='k')
pl.subplot(0, 1)
pl.add_mesh(mesh_sco_file, cmap="jet", show_scalar_bar=False, show_edges=True)
pl.add_text("Mesh of the stent", color='k')
pl.link_views()
pl.camera_position = 'iso'
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [103]:
u[0].name

'displacement_4.s'

In [110]:
# results from add_mesh method

file = "u_corr.vtk"
u_file = pyvista.read(file)
rotor3 = u_file.warp_by_scalar('U')
pl = pyvista.Plotter(notebook=True)
pl.add_mesh(u_file, scalars = 'U', scalar_bar_args={'title':'Displacement'}, color='blue')
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [115]:
# results from add_mesh method

file = "s_VM_corr.vtk"
s_VM_file = pyvista.read(file)
s_VM_file = s_VM_file.warp_by_scalar('S_VM')
pl = pyvista.Plotter(notebook=True)
pl.add_mesh(s_VM_file, scalars = "S_VM", scalar_bar_args={'title':'Displacement'}, color='blue')
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [117]:
# method using point_arrays because add_mesh method doesn't support point scalars

file = "u_stent_corr.vtk"
u_stent_file = pyvista.read(file)
u_stent_file.warp_by_scalar('U_STENT')
data = u_stent_file.get_array('U_STENT')
grid_vtk = mesh_sco.grid
grid_vtk.point_arrays['U_STENT'] = data
grid_vtk = grid_vtk.point_data_to_cell_data()
grid_vtk.plot(scalars='U_STENT')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
stop

NameError: name 'stop' is not defined

## Exit MAPDL

In [ ]:
mapdl.exit()